# From modules to packages

Note: We will be installing a couple of packages (`pytest`, `build`, `twine`) in this lesson, so it's best to do all this inside a new conda environment, not your system python or base conda environment. **Always use the latest pip, no exceptions!**

First, a tiny alias we define so we can clearly see folder structure. This is roughly equivalent to the 'tree' command in Linux.

In [ ]:
%alias tree find %s -print | sed -e 's;[^/]*/;|____;g;s;____|; |;g'

In [ ]:
%%bash

pip install --upgrade pip
pip install pytest build twine

The folder `00-walrus-package/` has the bare-minimum folder structure that we can respectably call a `package`. 

- It has the package source (of course).
- Our package has a version.
- Our package comes with tests.

In [ ]:
tree 00-walrus-package

Our package is called `walrus`. All its source is in a `src/` folder (Later on it will become clear why we have a `src/walrus/` folder and not just a `walrus/` folder), and tests inside the `test/` folder.

**What we want to hand over to our users is this entire `00-walrus-package/` folder, so they have our package source, as well as the tests to verify that our package is working correctly.**

So, how can we hand this over (*create a distribution*)?

- Zip up the `00-walrus-package/` folder and email it/put it up on the server.
- Point users to our Github page so they can clone our repo.
- Make everything (*the distribution archive*) available at a common marketplace - The Python Packaging Index (PyPI)

The last option seems most attractive (`pip install walrus` is as simple as it can get). **In addition, there are some HUGE advantages to it**:

- For complicated projects involving C/C++ code, we can do all the preparatory work (compilation/testing etc.) ahead of time such that `pip install walrus` works regardless of ones operating system or python version.
- For any other packages that our code depends on (*dependencies*, e.g. numpy/pandas/..), a `pip install walrus` will automatically install those dependencies for the user, and they don't have to worry about any of those details.

# Doing things (mostly) manually

In what follows, we'll be doing things largely manually so we can inspect files under the hood and understand what's going on. There are (arguably) easier ways to develop/upload packages, which we'll see soon.

## Testing our package under development

Our *tests* currently just ensure that our package has a version, so users can do a `import walrus; print(walrus.__version__);`. Let's make sure our tests pass.

In [ ]:
%%bash

cd 00-walrus-package/

# We don't HAVE the specify the folder containing our tests (test/)
# but we'll do so anyway to make it easier for pytest to do its job
pytest test

### What happened?

Of course python cannot find our package because it is buried inside the `src/` folder. Based on what we've learnt so far, let's see what we can do to fix the situation: (we'll be using `!` to run bash commands as a shortcut).

In [ ]:
!cd 00-walrus-package/ && pytest test

In [ ]:
!cd 00-walrus-package/ && python -c "import sys; print(sys.path)"

In [ ]:
!cd 00-walrus-package/ && PYTHONPATH=src python -c "import sys; print(sys.path)"

In [ ]:
!cd 00-walrus-package/ && PYTHONPATH=src pytest test

## Building our package

We'll be dealing with the `01-walrus-package/` folder from now on. Here we've added a `walrus.math.add_one` function, and added a test for it. We've also added three files - `pyproject.toml`, `setup.cfg`, and a `README.md`.

In [ ]:
tree 01-walrus-package

You may have seen other examples of packages involving a `setup.py`. Forget about those - `pyproject.toml` is the latest and greatest way of doing things (see [PEP517](https://www.python.org/dev/peps/pep-0517/)).

Essentially, it tells python to use a package called `setuptools` to start packaging the project. `setuptools` in turn looks at `setup.cfg` for some metadata about our package (name, version, where to find the source code etc). The `README.md` is for a longer description of our project that we've created just to be good citizens (`setuptools` likes it if we have it).

Go ahead and modify the package name/author/author_email/url in `setup.cfg`. *Why do you think I made the package name so complex instead of just using `walrus`*?

Internally, `pip` creates and activates a new virtual environment, installs `setuptools` and hands over control to it. `setuptools` builds our package, and creates two things for us (*artifacts*):

- The *source distribution* (`sdist`) - a `.tar.gz` file that represents all our source code.
- The *binary wheel* (or just *wheel*) - a `.whl` file. This is a compiled/optimized version, one for each platform and python version where we ran the whole build process.

Since in our case we have nothing but pure Python code (so no platform-specific stuff), the above two are mostly interchangeable. Note also that the wheel file is called `<name>-<version>-py3-none-any.whl` to indicate this platform/python-version free nature of our wheel.

**It is important that we always generate the *sdist***, so that our users (or rather `pip` running on their machines), can generate the `.whl` files on their machines, if they need to. The more `.whl` files we generate on our end and make available, the better of course.

For a much more detailed writeup on all this, see [An Overview of Packaging for Python](https://packaging.python.org/overview/)

In [ ]:
%%bash

cd 01-walrus-package
python -m build .

In [ ]:
tree 01-walrus-package

Let's open up these `.tar.gz` and `.whl` files and see what's actually inside them..

There's a `PKG_INFO`, some `egg` files etc. Nothing too complex, but not something that we would like to create by hand. Remember, **`setuptools` (and other newer, fancier ways of doing things like `poetry` or `flit`) make it easier to generate these sdists and wheel files, but in the end are doing the same thing.** Which one you choose is entirely a matter of taste.

## Installing our built package

Now that we have files in the `dist/` folder, we can install them using `pip`. Try it out!
    
1. `pip install <path/to/sdist>`
2. Uninstall the package we installed above (Hint: you could use `import walrus; print(walrus.__path__)` to see where it's installed, and `pip freeze` to see what the package is called so we can use `pip uninstall ..`
3. `pip install <path/to/whl>`
    
Notice how the second command runs much faster than the first one? Why?

(In fact, we could just have done `pip install .` to install our package instead of `python -m build .`, but we wanted to see what's really happening here.)

In [ ]:
%%bash

cd 01-walrus-package
pip install .

## Testing our installed package

Now that we have installed our package, let's see if our tests pass:

In [ ]:
%%bash

cd 01-walrus-package/
pytest test

This brings us to why we had a `src/` folder inside our source in the first place - before installing the package, `pytest test` would have failed, but now it passes. In general, **we would like to test the behavior of the package that eventually gets installed, not what we're developing (certainly our users would be concerned about the behavior of the installed package)**. Having a `src/` folder prevents us from *accidentally* picking up our source folder during development, unless we're very explicit about it (by specifying `PYTHONPATH` etc.)

## Making the package available on PyPI

Now that the distribution artifacts are built in `dist/` (and tested!), we can upload them to [PyPI](https://pypi.org/) using `twine`. We'll have to create an account on PyPI first of course.

In [ ]:
%%bash

cd 01-walrus-package/
twine upload dist/*

# Packaging data along with your code

There are use cases where you may want to package data files (.csv files, pre-trained ML models etc.) so that someone running a `pip install` automatically gets those file (and further, these files can easily be used by the code). One excellent reason you may want to include data files is to provide small, minimal sample files that illustrate how to use your package, and tests and sample scripts that do actually use these files.

The exact procedure you take will depend on which *build backend* you use (as per your `pyproject.toml`). Since so far we've used `setuptools` (the most common), let's see how this works with `setuptools`.

Here are the key points:

- `setuptools` packaged a few things in our `sdist` (and the `wheel`) automatically for us when we said `python -m build .` and we didn't have to specify anything. The exact rules it follows are [here](https://packaging.python.org/guides/using-manifest-in/#how-files-are-included-in-an-sdist)

- For more fine-grained control over what gets included, we can use a file called `MANIFEST.in`. This explicitly lists *commands* that tell `setuptools` which files should be included or excluded after `setuptools` has done its default processing. There are [quite a few](https://packaging.python.org/guides/using-manifest-in/#manifest-in-commands) commands, but we generally can get away with `graft` or `recursive-include`.

- A `MANIFEST.in` that looks like the following will pull in all files from the `src/walrus/data` directory and make them available in the `sdist`.

```
recursive-include src/walrus/data *
```
    

- In addition, we'll also add an `include_package_data = True` in our `setup.cfg` to tell `setuptools` to also include it in the built (binary) distribution.

- To access the data programmatically from our package code, we'll make the `data` folder a `module` (by including a blank `__init__.py`), and use functions from `importlib.resources` (a built-in python module for python >= 3.7) to read it.

  You can choose to use `open` or some hack with `__file__` in your code to find out where your data file is, but this will result in **fragile code that will likely break over time.** (Consider - what if your whole package was zipped up?) `importlib.resources` uses the same mechanism that python uses to load modules, and will always work, now and in the future. Also, remember - you only have to use `importlib.resources` to access data that you pre-package. You can use `open`/`shutil` etc. in functions that you expose to your users to read files at runtime.
  
- See the `02-walrus-package/` folder for a complete example.

# Specifying dependencies for your package

If our package in turn depends on other packages, we specify it as a [required dependency](https://setuptools.readthedocs.io/en/latest/userguide/dependency_management.html#declaring-required-dependency) in the `install_requires` section of `setup.cfg`. See `03-walrus-package/` as an example, which depends on `pandas`.

If it's possible for users to use our package but perhaps with reduced functionality, we can choose to declare a dependency as [optional](https://setuptools.readthedocs.io/en/latest/userguide/dependency_management.html#optional-dependencies). In the case of `03-walrus-package/`:

```
    [options.extras_require]
        movies = pandas
```

A `pip install walrus` will install our package and its required dependencies, but a `pip install walrus[movies]` will install the optional dependencies under the `movies` section above, allowing the user to use the `walrus.movies` submodule.

Of course, our code must be written in such a way that it's capable of working under the absence of optional dependencies (you would not want to `import pandas` while trying to load your package, by either moving those `import` statements as deep as possible (not recommended) or guarding them against `ImportError`s (recommended).


Optional dependencies are typically used to allow other developers to install your package along with other dependencies that allow them to continue development on their machines (e.g. a `pip install -e .[dev]` might also install `pytest`, `sphinx` etc.)